In [1]:
import import_ipynb #allows access to import other notebooks
from PreprocessLib import cleanText, removeSingleLists, process_file

importing Jupyter notebook from PreprocessLib.ipynb


In [2]:
#returns file in string format
def stringify(a_file):
    string_without_line_breaks = ""
    for line in a_file:
        stripped_line = line.rstrip() + " "
        string_without_line_breaks += stripped_line
    a_file.close()
    sentences = string_without_line_breaks
    return sentences

In [3]:
clemFile = open("./finalTexts/clement.txt", "r", encoding="utf8")
lukeFile = open("./finalTexts/luke.txt", "r", encoding="utf8")
pastFile = open("./finalTexts/pastoral.txt", "r", encoding="utf8")
paulFile = open("./finalTexts/paul.txt", "r", encoding="utf8")
testFile = open("./finalTexts/hebrews.txt", "r", encoding="utf8")
clemFreq = process_file(clemFile, False)
lukeFreq = process_file(lukeFile, False)
pastFreq = process_file(pastFile, False)
paulFreq = process_file(paulFile, False)
testFreq = process_file(testFile, False)

clemFile = open("./finalTexts/clement.txt", "r", encoding="utf8")
lukeFile = open("./finalTexts/luke.txt", "r", encoding="utf8")
pastFile = open("./finalTexts/pastoral.txt", "r", encoding="utf8")
paulFile = open("./finalTexts/paul.txt", "r", encoding="utf8")
testFile = open("./finalTexts/hebrews.txt", "r", encoding="utf8")
#sentence tokenizes the documents
clem = stringify(clemFile)
luke = stringify(lukeFile)
past = stringify(pastFile)
paul = stringify(paulFile)
test = stringify(testFile)
from nltk import tokenize
clemSent = tokenize.sent_tokenize(clem)
lukeSent = tokenize.sent_tokenize(luke)
pastSent = tokenize.sent_tokenize(past)
paulSent = tokenize.sent_tokenize(paul)
testSent = tokenize.sent_tokenize(test)

In [4]:
print(len(clemFreq.split()))
print(len(lukeFreq.split()))
print(len(pastFreq.split()))
print(len(paulFreq.split()))
print(len(testFreq.split()))

10233
37949
3492
24121
4955


In [5]:
#uses a g-test, useful for finding differences in proportions
#returns signifcance of G-Test
def isSignificant(obA, exA, obB, exB):
    import numpy as np
    from scipy.stats import chi2_contingency
    obs = np.array([[obA, exA-obA], [obB, exB-obB]])
    p = chi2_contingency(obs, lambda_="log-likelihood")
    pVal = p[1] #p[1] contains the p-value of the test
   # print(pVal)
    if (pVal < 0.05):
        return True
    else:
        return False

In [8]:
import re
from operator import itemgetter
import numpy as np
from scipy.stats import chi2_contingency
#returns list of instances and count in sorted order from most to least common
def mostCommon(instances):
    return sorted(sorted(Counter(instances).items(), key=itemgetter(0)), key=itemgetter(1), reverse=True)

#returns frequency of word in txt
def getFreq(word, txt):
    return txt.count(word)

#analyzes frequency of the words in the document
#returns most common words of each document and count of significantly different word frequencies
# and each iteration counts which authors words was most similar, then returns the list
def analyzeFreq():
    from scipy import stats
    from collections import Counter
    clemList = re.sub("[^\w]", " ",  clemFreq).split()
    clemCounts = Counter(clemList)
    lukeList = re.sub("[^\w]", " ",  lukeFreq).split()
    lukeCounts = Counter(lukeList)
    pastList = re.sub("[^\w]", " ",  pastFreq).split()
    pastCounts = Counter(pastList)
    paulList = re.sub("[^\w]", " ",  paulFreq).split()
    paulCounts = Counter(paulList)
    testList = re.sub("[^\w]", " ",  testFreq).split()
    testCounts = Counter(testList)

    print(clemCounts.most_common(10))
    print(lukeCounts.most_common(10))
    print(pastCounts.most_common(10))
    print(paulCounts.most_common(10))
    print(testCounts.most_common(10))
    
    countList = []
    Cl = 0
    Lu = 0
    Pa = 0
    Pu = 0
    wordCount = 0
    
    for word in testCounts:
        label = 'notinany'
        minDiff = 99999 
        count = 0
        testWordFreq = getFreq(word, testFreq)

        if (word in clemCounts):
            tempFreq = getFreq(word, clemFreq)
            if (isSignificant(tempFreq, len(clemList), testWordFreq, len(testList))):
                Cl += 1
            tempDiff = abs(clemCounts[word] - testCounts[word])
            count += 1
            if (tempDiff < minDiff):
                label = "Clement"
                minDiff = tempDiff
        if (word in lukeCounts):
            tempFreq = getFreq(word, lukeFreq)
            if (isSignificant(tempFreq, len(lukeList), testWordFreq, len(testList))):     
                Lu += 1
            tempDiff = abs(lukeCounts[word] - testCounts[word])
            count += 1
            if (tempDiff < minDiff):
                label = "Luke"
                minDiff = tempDiff
        if (word in pastCounts):
            tempFreq = getFreq(word, pastFreq)
            if (isSignificant(tempFreq, len(pastList), testWordFreq, len(testList))):     
                Pa += 1
            tempDiff = abs(pastCounts[word] - testCounts[word])
            count += 1
            if (tempDiff < minDiff):
                label = "Pastoral"
                minDiff = tempDiff
        if (word in paulCounts):
            tempFreq = getFreq(word, paulFreq)
            if (isSignificant(tempFreq, len(paulList), testWordFreq, len(testList))):     
                Pu += 1
            tempDiff = abs(paulCounts[word] - testCounts[word])
            count += 1
            if (tempDiff < minDiff):
                label = "Paul"
                minDiff = tempDiff
        if (label != 'notinany' and count > 3):
            countList.append(label)
            wordCount += 1
    from collections import Counter
    from collections import OrderedDict
    list = Counter(countList)
    print(OrderedDict([(i, str(round(count / sum(list.values()) * 100.0, 3)) + '%') for i, count in list.most_common()]))
    print(Cl, Lu, Pa, Pu)
    print(Cl/wordCount, Lu/wordCount, Pa/wordCount, Pu/wordCount)
    
    

In [9]:
analyzeFreq()

[('καὶ', 677), ('ἐν', 186), ('τοῦ', 169), ('αὐτοῦ', 165), ('τῆς', 143), ('τὸ', 134), ('εἰς', 116), ('ὁ', 115), ('τῶν', 113), ('τὴν', 95)]
[('καὶ', 2560), ('δὲ', 1034), ('τοῦ', 707), ('ὁ', 707), ('ἐν', 635), ('εἰς', 527), ('τὸν', 489), ('τὸ', 422), ('τὴν', 403), ('αὐτοῦ', 384)]
[('καὶ', 196), ('ἐν', 94), ('δὲ', 58), ('τοῦ', 46), ('τὴν', 42), ('εἰς', 39), ('μὴ', 39), ('τῆς', 37), ('ὁ', 37), ('θεοῦ', 35)]
[('καὶ', 1051), ('ἐν', 674), ('δὲ', 493), ('τοῦ', 454), ('τὸ', 377), ('ὁ', 361), ('γὰρ', 356), ('εἰς', 316), ('τῶ', 263), ('μὴ', 251)]
[('καὶ', 255), ('γὰρ', 87), ('τῆς', 85), ('εἰς', 74), ('τὸν', 74), ('ὁ', 69), ('τὴν', 69), ('τὸ', 68), ('δὲ', 67), ('τοῦ', 67)]
OrderedDict([('Pastoral', '50.427%'), ('Clement', '40.171%'), ('Luke', '6.838%'), ('Paul', '2.564%')])
57 124 37 113
0.24358974358974358 0.5299145299145299 0.1581196581196581 0.4829059829059829


In [10]:
#POS tags the documents
from angel import tag
clemTags = tag(clem)
lukeTags = tag(luke)
pastTags = tag(past)
paulTags = tag(paul)
testTags = tag(test)

Loading models...
Part-of-speech models loading...
Person models loading...
Number models loading...
Tense models loading...
Mood models loading...
Voice models loading...
Gender models loading...
Case models loading...
Degree models loading...
Pre-processing text...
Text and punctuation split into 12563 individual tokens.
Angel's looking at each word by itself...
Reconsidering tags...
Studying each word in light of its context...
Loading models...
Part-of-speech models loading...
Person models loading...
Number models loading...
Tense models loading...
Mood models loading...
Voice models loading...
Gender models loading...
Case models loading...
Degree models loading...
Pre-processing text...
Text and punctuation split into 42423 individual tokens.
Angel's looking at each word by itself...
Reconsidering tags...
Studying each word in light of its context...
Loading models...
Part-of-speech models loading...
Person models loading...
Number models loading...
Tense models loading...
Mood 

In [11]:
#takes in  list of all tagLists "tags", a word to look for "testCase", and the index of the POS tag of interest
#returns frequencies of testCase in each document, and G-test result of the frequencies with the test frequencies
def freqOf(tags, testCase, index):
    results = []
    for tagList in tags:
        counter = 0
        for word in tagList:
            if (word[1][index] == testCase):
                counter += 1
        results.append(counter)
    
    print(results)
    i = 0
    for case in results[:-1]:
        print("Testing index ", i, " with test:")
        print("Frequencies: ", round(case/len(tags[i]), 5), " ", round(results[4]/len(tags[4]),5))
        if (case == 0):
            case = 0.000000000000000000001 #if case stays 0, there is an error with the g-test
        if (isSignificant(case, len(tags[i]), results[4], len(tags[4]))):
            print("Different")
        else:
            print("Not Different")
        print('\n')
        i += 1

In [12]:
#testing frequency of optative mood usage 
tags = [clemTags, lukeTags, pastTags, paulTags, testTags]
indexMood = 4
freqOf(tags, "o", indexMood)

[8, 29, 2, 23, 0]
Testing index  0  with test:
Frequencies:  0.00064   0.0
Not Different


Testing index  1  with test:
Frequencies:  0.00068   0.0
Different


Testing index  2  with test:
Frequencies:  0.00049   0.0
Not Different


Testing index  3  with test:
Frequencies:  0.00084   0.0
Different




In [76]:
#testing frequency of future perfect tense usage
indexTense = 3
freqOf(tags, "t", indexTense)

[0, 0, 0, 0, 0]
Testing index  0  with test:
Frequencies:  0.0   0.0
1.0
Not Different


Testing index  1  with test:
Frequencies:  0.0   0.0
1.0
Not Different


Testing index  2  with test:
Frequencies:  0.0   0.0
1.0
Not Different


Testing index  3  with test:
Frequencies:  0.0   0.0
1.0
Not Different




In [14]:
#testing frequency of pluperfect tense usage
freqOf(tags, "l", indexTense)

[6, 39, 3, 7, 4]
Testing index  0  with test:
Frequencies:  0.00048   0.00072
Not Different


Testing index  1  with test:
Frequencies:  0.00092   0.00072
Not Different


Testing index  2  with test:
Frequencies:  0.00074   0.00072
Not Different


Testing index  3  with test:
Frequencies:  0.00026   0.00072
Not Different




In [24]:
#takes in a tagList and returns counts of participles before or after main verb
def verbCheck(tagList):
    leftCount = 0
    rightCount = 0
    mainVerbFound = False
    for word in tagList:
        if ((word[1][4] == 'i' or word[1][4] == 'm') and word[1][0] == 'v' and mainVerbFound == False):
            mainVerbFound = True
            leftCount += 1
        else:
            rightCount += 1
    return (leftCount, rightCount)

In [60]:
splitWords = ["καί", "δέ", "ἀλλά", "τε", "ἄρα", "διό"]
#takes in a text and a tag list, returns count of participles before and after main verb
def participleAnalysis(txt, tagList):
    import string
    j = 0
    leftCount = 0
    rightCount = 0
    table = str.maketrans(dict.fromkeys(string.punctuation)) 
    for sent in txt:
        sentenceSplit = sent.split()
       # print("sent: ", sentenceSplit)
        firstHalf = []
        secondHalf = []
        enterSecond = False
        for word in sentenceSplit:
            if (tagList[j][0] in string.punctuation):
                j += 1
            testWord = word.translate(table)
            line = testWord
            testWord = ''.join( c for c in line if  c not in '’᾿·0123456789' ) #to fix a bug with greek punctuation
            testWord2 = tagList[j][0].translate(table)
            line = testWord2
            testWord2 = ''.join( c for c in line if  c not in '’᾿·0123456789' )
            if (len(testWord2) < 1): #to fix a bug with multiple spaces present in word
                j += 2
            if (testWord == testWord2):
                firstHalf.append(tagList[j])
                if (testWord in splitWords and len(firstHalf) > 3 and enterSecond == False):
                    enterSecond = True
                j += 1
            if (enterSecond == True):
                secondHalf.append(tagList[j])
        if (enterSecond == True):
            count = verbCheck(firstHalf)
            leftCount += count[0]
            rightCount += count[1]
            count = verbCheck(secondHalf)
            leftCount += count[0]
            rightCount += count[1]
        if (enterSecond == False):
            count = verbCheck(firstHalf)
            leftCount += count[0]
            rightCount += count[1]
    print(leftCount)
    print(rightCount)

In [71]:
participleAnalysis(clemSent, clemTags)
isSignificant(366,10347,161,5324)

366
10347
0.0979999974185213


False

In [72]:
participleAnalysis(lukeSent, lukeTags)
isSignificant(1635,38760,161,5324)

1635
38760
2.0513469283665826e-05


True

In [73]:
participleAnalysis(pastSent, pastTags)
isSignificant(131,3397,161,5324)

131
3397
0.04226510799143014


True

In [74]:
participleAnalysis(paulSent, paulTags)
isSignificant(898,23984,161,5324)

898
23984
0.010443997278349841


True

In [75]:
participleAnalysis(testSent, testTags)

161
5324
